In [2]:
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from sklearn.model_selection import KFold

In [23]:
df = pd.read_csv('data/preprocessed_submission_fin.csv', encoding='ISO-8859-1')
df.head()

,id,bant_submit,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,...,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,19844,0.00,ID,0.073248,47466,End Customer,Enterprise,53,0,0,...,LGESP,Others,1,0,0.001183,0.049840,retail,Electronics & Telco,278,True
1,9738,0.25,IT,0.102336,5405,End Customer,SMB,0,0,0,...,LGEUS,Others,0,0,0.000013,0.058388,transportation,Others,437,True
2,8491,1.00,ID,0.102336,13597,Specifier/ Influencer,SMB,0,0,0,...,LGEGF,less than 3 months,0,0,0.000060,0.131148,hospital & health care,General Hospital,874,False
3,19895,0.50,ID,0.118644,17204,Unknown,Enterprise,0,0,0,...,LGEUS,more than a year,0,0,0.001183,0.049840,retail,NaN,194,False
4,10465,1.00,ID,0.074949,2329,End Customer,Enterprise,2,1,0,...,LGESP,less than 3 months,1,1,0.003079,0.064566,corporate / office,Engineering,167,False


In [4]:
print(len(df.columns))
df.columns

28


Index(['bant_submit', 'business_unit', 'com_reg_ver_win_rate', 'customer_idx',
       'customer_type', 'enterprise', 'historical_existing_cnt',
       'id_strategic_ver', 'it_strategic_ver', 'idit_strategic_ver',
       'customer_job', 'lead_desc_length', 'inquiry_type', 'product_category',
       'product_subcategory', 'product_modelname', 'customer_country.1',
       'customer_position', 'response_corporate', 'expected_timeline',
       'ver_cus', 'ver_pro', 'ver_win_rate_x', 'ver_win_ratio_per_bu',
       'business_area', 'business_subarea', 'lead_owner', 'is_converted'],
      dtype='object')

In [5]:
# categorical columns

for col in df.columns:
    if df[col].dtype != 'int64' and df[col].dtype != 'float64':
        print(col)

business_unit
customer_type
enterprise
customer_job
inquiry_type
product_category
product_subcategory
product_modelname
customer_country.1
customer_position
response_corporate
expected_timeline
business_area
business_subarea
is_converted


In [6]:
#numerical columns

for col in df.columns:
    if df[col].dtype == 'int64' or df[col].dtype == 'float64':
        print(col)

bant_submit
com_reg_ver_win_rate
customer_idx
historical_existing_cnt
id_strategic_ver
it_strategic_ver
idit_strategic_ver
lead_desc_length
ver_cus
ver_pro
ver_win_rate_x
ver_win_ratio_per_bu
lead_owner


In [10]:
# inquiry_type

# count how many na in inquiry_type
print(df['inquiry_type'].isna().sum())
print(len(df['inquiry_type'].unique()))
print(df['inquiry_type'].value_counts())

print(df['inquiry_type'].unique())

891
72
inquiry_type
Quotation or purchase consultation                                                                                           22908
Quotation or Purchase Consultation                                                                                           17228
Sales Inquiry                                                                                                                 8844
Product Information                                                                                                           1100
Other                                                                                                                          901
                                                                                                                             ...  
quotation_                                                                                                                       1
display product                                                

In [20]:
# Replace the specified 'expected_timeline' values
df['inquiry_type'] = df['inquiry_type'].replace({
    'Quotation or Purchase Consultation': 'Quotation or purchase consultation',  # merge into 'less than 3 months'
    'quotation_or_purchase_consultation': 'Quotation or purchase consultation',  # merge into 'being followed up.'
    'quotation_':'Quotation or purchase consultation',
    'Quotation or Purchase consultation':'Quotation or purchase consultation',
    'Purchase or Quotation':'Quotation or purchase consultation',
    'Purchase' : 'Quotation or purchase consultation',
    'Usage or Technical Consultation':'Usage or technical consultation',
    'Technical Consultation':'Usage or technical consultation',
    'technical':'Usage or technical consultation',
    'usage or technical consultation' : 'Usage or technical consultation',
    'usage_or_technical_consultation':'Usage or technical consultation',
    'Technical Support':'Usage or technical consultation',
    'technical_consultation':'Usage or technical consultation',
    'Sales Inquiry':'Sales inquiry',
    'sales':'Sales inquiry',
    'estoy buscando para Ecuador este producto LG MAGNIT micro LED, para un cliente de 138 pulgadas, con enviÃ\x83Â³ marÃ\x83Â\xadtimo.':'Product inquiry',
    'Hola me pueden cotizar 19 pantallas interactivas de 100 pulgadas entregadas en Guayaquil -Ecuador.': 'Product inquiry',
    'window facing product':'Product inquiry',
    'Vui lÃ\x83Â²ng bÃ\x83Â¡o giÃ\x83Â¡ giÃ\x83Âºp mÃ\x83Â¬nh sÃ¡ÂºÂ£n phÃ¡ÂºÂ©m Ã\x84Â\x91o thÃ\x83Â¢n nhiÃ¡Â»Â\x87t Xin cÃ¡ÂºÂ£m Ã\x86Â¡n':'Product inquiry',
    'Hotel TV products':'Product inquiry',
    'Probeam precio':'Product inquiry',
    'Pantallas Interactivas para Clinicas':'Product inquiry',
    'Solicito apoyo para realizar cotizacion de los dispositivos que ofrecen en la soluciÃ\x83Â³nÃ\x82Â\xa0One Quick:Ã\x82Â\xa0':'Product inquiry',
    'IntÃ\x83Â©grateur historique du George V':'Product inquiry',
    'IDB':'Product inquiry',
    'for school':'Product inquiry',
    'LED Signage':'Product inquiry',
    'Standalone':'Product inquiry',
    'Video Wall':'Product inquiry',
    'Toi muon tim hieu thong tin ky thuat, gia ca cua sp de su dung':'Product inquiry',
    'tÃ\x83Â´i cÃ¡ÂºÂ§n tham khÃ¡ÂºÂ£o giÃ\x83Â¡ vÃ\x83Â\xa0 giÃ¡ÂºÂ£i phÃ\x83Â¡p tÃ¡Â»Â« LG':'Product inquiry',
    'Preciso de um monitor mÃ\x83Â©dico para radiografia convencional e tomogrtafia.': 'Product inquiry',
    'VRF':'Product inquiry',
    'Evento_SdelEstero':'Product inquiry',
    'Product Information':'Product inquiry',
    'AIO':'Product inquiry',
    'Hospital TV':'Product inquiry',
    'TV interactive':'Product inquiry',
    'Display Textbook and photos':'Product inquiry',
    'Others':'Other',
    'others':'Other',
    'other':'Other',
    'ETC.': 'Other',
    'Etc.': 'Other',
    'other_':'Other',
    'High inch 86 / 98 or 110':'Product inquiry',
    'EDUCATIONAL EQUIPMENTS':'Product inquiry',
    'teach':'Product inquiry',
    'Not specified':'Other',
    'One Quick:Flex':'Product inquiry',
    'Digital platform':'Product inquiry',
    'display product':'Product inquiry',
    'Request for technical consulting':'Usage or technical consultation',
    'Request for quotation or purchase':'Quotation or purchase consultation',
    'Request a Demo':'Usage or technical consultation',
    'Request for Distributorship':'Sales inquiry',
    'Request for Partnership':'Sales inquiry',
    'Event Inquiry':'Sales inquiry',
    'Media Inquiry':'Product inquiry',
    'Needs':'Other',
    '(Select ID_Needs)':'Other',
    'OEM/ODM Request':'Sales inquiry',
    'i want to know the details about it':'Other',
    'first Info and pricing':'Product inquiry'
})

# Optional: Check the updated value counts to confirm the merges
updated_counts = df['inquiry_type'].value_counts()
print(len(updated_counts))
print(updated_counts)

8
inquiry_type
Quotation or purchase consultation    40468
Sales inquiry                          9297
Usage or technical consultation        1864
Other                                  1368
Product inquiry                        1143
Trainings                               376
Services                                361
Customer Suggestions                     12
Name: count, dtype: int64


## customer_country.1

In [6]:
len(df['customer_country.1'].unique())

405

In [7]:
df_encoded = pd.get_dummies(df, columns=['customer_country.1'])

# Show the first few rows to verify the new one-hot encoded columns
print(df_encoded.head())

   bant_submit business_unit  com_reg_ver_win_rate  customer_idx  \
0          1.0            AS              0.066667         32160   
1          1.0            AS              0.066667         23122   
2          1.0            AS              0.088889          1755   
3          1.0            AS              0.088889          4919   
4          1.0            AS              0.088889         17126   

           customer_type  enterprise  historical_existing_cnt  \
0           End-Customer  Enterprise                        0   
1           End-Customer  Enterprise                       12   
2           End-Customer  Enterprise                      144   
3           End-Customer  Enterprise                        0   
4  Specifier/ Influencer  Enterprise                        0   

   id_strategic_ver  it_strategic_ver  idit_strategic_ver  ...  \
0                 0                 0                   0  ...   
1                 0                 0                   0  ...   
2 

## inquiry_type

In [314]:
df['inquiry_type'].unique()

array([nan, 'Quotation or Purchase Consultation', 'Product Information',
       'Technical Consultation', 'Others', 'Request for Partnership',
       'Usage or Technical Consultation', 'Services',
       'Customer Suggestions', 'Request a Demo', 'Trainings', 'Other',
       'OEM/ODM Request', 'Media Inquiry', 'other_',
       'Request for Distributorship'], dtype=object)

In [10]:
# Apply one-hot encoding to 'customer_country.1'
df_encoded = pd.get_dummies(df, columns=['inquiry_type'])

# Show the first few rows to verify the new one-hot encoded columns
print(df_encoded.head())

KeyError: "None of [Index(['inquiry_type'], dtype='object')] are in the [columns]"

## business_unit

In [8]:
df.business_unit.unique()

array(['AS', 'ID', 'IT', 'Solution', 'CM'], dtype=object)

In [9]:
# Apply one-hot encoding to 'customer_country.1'
df_encoded = pd.get_dummies(df, columns=['business_unit'])

# Show the first few rows to verify the new one-hot encoded columns
print(df_encoded.head())

   bant_submit  com_reg_ver_win_rate  customer_idx          customer_type  \
0          1.0              0.066667         32160           End-Customer   
1          1.0              0.066667         23122           End-Customer   
2          1.0              0.088889          1755           End-Customer   
3          1.0              0.088889          4919           End-Customer   
4          1.0              0.088889         17126  Specifier/ Influencer   

   enterprise  historical_existing_cnt  id_strategic_ver  it_strategic_ver  \
0  Enterprise                        0                 0                 0   
1  Enterprise                       12                 0                 0   
2  Enterprise                      144                 0                 0   
3  Enterprise                        0                 0                 0   
4  Enterprise                        0                 0                 0   

   idit_strategic_ver             customer_job  ...  ver_win_ratio_p

In [319]:
df['business_unit'] = df['business_unit_te']
df.drop('business_unit_te', axis=1, inplace=True)
print(len(df.columns))
df.columns

30


Index(['id', 'bant_submit', 'customer_country', 'business_unit',
       'com_reg_ver_win_rate', 'customer_idx', 'customer_type', 'enterprise',
       'historical_existing_cnt', 'id_strategic_ver', 'it_strategic_ver',
       'idit_strategic_ver', 'customer_job', 'lead_desc_length',
       'inquiry_type', 'product_category', 'product_subcategory',
       'product_modelname', 'customer_country.1', 'customer_position',
       'response_corporate', 'expected_timeline', 'ver_cus', 'ver_pro',
       'ver_win_rate_x', 'ver_win_ratio_per_bu', 'business_area',
       'business_subarea', 'lead_owner', 'is_converted'],
      dtype='object')

## Customer_Type

### 5-fold Target Encoding

In [320]:
print(df.customer_type.unique())
len(df.customer_type.unique())

['End Customer' 'Specifier/ Influencer' 'Unknown' 'Service Partner'
 'Channel Partner' 'Solution Eco-Partner' 'Developer' 'End-Customer']


8

In [321]:
# Initialize KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Create a new column for the target-encoded 'customer_type'
df['customer_type_te'] = np.nan

# Apply K-fold target encoding
for train_idx, val_idx in kf.split(df):
    # Split the data
    df_train, df_val = df.iloc[train_idx], df.iloc[val_idx]
    # Compute the target mean for each category in 'customer_type' on the training data
    means = df_train.groupby('customer_type')['is_converted'].mean()
    # Map the computed means to the validation set
    df.loc[val_idx, 'customer_type_te'] = df_val['customer_type'].map(means)

# Fill missing values for any 'customer_type' not seen in the training folds
global_mean = df['is_converted'].mean()
df['customer_type_te'].fillna(global_mean, inplace=True)

In [322]:
# replace the original column with the target encoded column, but the order matters

df['customer_type'] = df['customer_type_te']
df.drop('customer_type_te', axis=1, inplace=True)
print(len(df.columns))
df.columns

30


Index(['id', 'bant_submit', 'customer_country', 'business_unit',
       'com_reg_ver_win_rate', 'customer_idx', 'customer_type', 'enterprise',
       'historical_existing_cnt', 'id_strategic_ver', 'it_strategic_ver',
       'idit_strategic_ver', 'customer_job', 'lead_desc_length',
       'inquiry_type', 'product_category', 'product_subcategory',
       'product_modelname', 'customer_country.1', 'customer_position',
       'response_corporate', 'expected_timeline', 'ver_cus', 'ver_pro',
       'ver_win_rate_x', 'ver_win_ratio_per_bu', 'business_area',
       'business_subarea', 'lead_owner', 'is_converted'],
      dtype='object')

## Enterprise

### One-Hot Encoding

In [323]:
print(df.enterprise.unique())
len(df.enterprise.unique())

df.enterprise.head()

['Enterprise' 'SMB']


0    Enterprise
1           SMB
2           SMB
3    Enterprise
4    Enterprise
Name: enterprise, dtype: object

In [324]:
df_one_hot = pd.get_dummies(df, columns=['enterprise'], prefix='enterprise', drop_first=False)

# The drop_first=True option drops the first category column, avoiding multicollinearity
# 'prefix' is used to name the new column(s) based on the original column name for clarity

# # Display the first few rows to verify the new one-hot encoded column
print(df_one_hot['enterprise_Enterprise'].head())
print(df_one_hot['enterprise_SMB'].head())

0     True
1    False
2    False
3     True
4     True
Name: enterprise_Enterprise, dtype: bool
0    False
1     True
2     True
3    False
4    False
Name: enterprise_SMB, dtype: bool


In [325]:
df = df_one_hot
print(len(df.columns))
df.columns

31


Index(['id', 'bant_submit', 'customer_country', 'business_unit',
       'com_reg_ver_win_rate', 'customer_idx', 'customer_type',
       'historical_existing_cnt', 'id_strategic_ver', 'it_strategic_ver',
       'idit_strategic_ver', 'customer_job', 'lead_desc_length',
       'inquiry_type', 'product_category', 'product_subcategory',
       'product_modelname', 'customer_country.1', 'customer_position',
       'response_corporate', 'expected_timeline', 'ver_cus', 'ver_pro',
       'ver_win_rate_x', 'ver_win_ratio_per_bu', 'business_area',
       'business_subarea', 'lead_owner', 'is_converted',
       'enterprise_Enterprise', 'enterprise_SMB'],
      dtype='object')

## customer_job

### 5fold Target Encoding

In [326]:
print(df.customer_job.unique())
len(df.customer_job.unique())

['consulting' nan 'information technology' 'sales' 'engineering'
 'marketing' 'purchasing' 'others' 'operations' 'support' 'finance'
 'business development' 'administrative' 'education' 'product management'
 'program and project management' 'entrepreneurship'
 'community and social services' 'media and communication'
 'arts and design' 'healthcare services' 'accounting'
 'medical imaging specialist' 'clinical specialist' 'curation' 'other'
 '3d/vfx art' 'legal' 'radiology professional'
 'military and protective services' 'real estate' 'human resources'
 'healthcare' 'electronics & telco' 'quality assurance' 'pathologist'
 'research' 'surgery professional' 'film production' 'k12 school'
 'media and communications' 'medical solution provider']


42

In [327]:
# Initialize KFold
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Create a new column for the target-encoded 'customer_type'
df['customer_job_te'] = np.nan

# Apply K-fold target encoding
for train_idx, val_idx in kf.split(df):
    # Split the data
    df_train, df_val = df.iloc[train_idx], df.iloc[val_idx]
    # Compute the target mean for each category in 'customer_type' on the training data
    means = df_train.groupby('customer_job')['is_converted'].mean()
    # Map the computed means to the validation set
    df.loc[val_idx, 'customer_job_te'] = df_val['customer_job'].map(means)

# Fill missing values for any 'customer_type' not seen in the training folds
global_mean = df['is_converted'].mean()
df['customer_job_te'].fillna(global_mean, inplace=True)

In [328]:
df['customer_job_te'].head()

0    0.255034
1    0.221969
2    0.271429
3    0.297297
4    0.183486
Name: customer_job_te, dtype: float64

In [329]:
df['customer_job'] = df['customer_job_te']
df.drop('customer_job_te', axis=1, inplace=True)
print(len(df.columns))
df.columns

31


Index(['id', 'bant_submit', 'customer_country', 'business_unit',
       'com_reg_ver_win_rate', 'customer_idx', 'customer_type',
       'historical_existing_cnt', 'id_strategic_ver', 'it_strategic_ver',
       'idit_strategic_ver', 'customer_job', 'lead_desc_length',
       'inquiry_type', 'product_category', 'product_subcategory',
       'product_modelname', 'customer_country.1', 'customer_position',
       'response_corporate', 'expected_timeline', 'ver_cus', 'ver_pro',
       'ver_win_rate_x', 'ver_win_ratio_per_bu', 'business_area',
       'business_subarea', 'lead_owner', 'is_converted',
       'enterprise_Enterprise', 'enterprise_SMB'],
      dtype='object')

## product_category

### 5fold TE

In [330]:
print(df.product_category.unique())
len(df.product_category.unique())

[nan 'hospital tv' 'led signage' 'cloud device' 'standard signage'
 'interactive signage' 'multi-split' 'video wall signage' 'vrf' 'monitor'
 'etc.' 'heating' 'ventilation' 'control' 'single-split'
 'high brightness signage' 'chiller' 'oled signage' 'special signage'
 'hotel tv' 'projector' 'medical display' 'signage care solution' 'ess'
 'laptop' 'interactive digital board' 'pro:centric' 'software solution'
 'others' 'one:quick series' 'webos' 'notebook' 'accessories' 'other']


34

In [331]:
df.product_category.value_counts()
# print unique values and their value counts when the value count is more than 1000
df.product_category.value_counts()[df.product_category.value_counts() > 1000]

Series([], Name: count, dtype: int64)

In [332]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Create a new column for the target-encoded 'product_category'
df['product_category_te'] = np.nan

# Apply K-fold target encoding
for train_idx, val_idx in kf.split(df):
    # Split the data
    df_train, df_val = df.iloc[train_idx], df.iloc[val_idx]
    # Compute the target mean for each category in 'product_category' on the training data
    means = df_train.groupby('product_category')['is_converted'].mean()
    # Map the computed means to the validation set
    df.loc[val_idx, 'product_category_te'] = df_val['product_category'].map(means)

# Fill missing values for any 'product_category' not seen in the training folds
global_mean = df['is_converted'].mean()
df['product_category_te'].fillna(global_mean, inplace=True)

In [333]:
df['product_category'] = df['product_category_te']
df.drop('product_category_te', axis=1, inplace=True)
print(len(df.columns))
print(df.columns)
df['product_category'].head()

31
Index(['id', 'bant_submit', 'customer_country', 'business_unit',
       'com_reg_ver_win_rate', 'customer_idx', 'customer_type',
       'historical_existing_cnt', 'id_strategic_ver', 'it_strategic_ver',
       'idit_strategic_ver', 'customer_job', 'lead_desc_length',
       'inquiry_type', 'product_category', 'product_subcategory',
       'product_modelname', 'customer_country.1', 'customer_position',
       'response_corporate', 'expected_timeline', 'ver_cus', 'ver_pro',
       'ver_win_rate_x', 'ver_win_ratio_per_bu', 'business_area',
       'business_subarea', 'lead_owner', 'is_converted',
       'enterprise_Enterprise', 'enterprise_SMB'],
      dtype='object')


0    0.221969
1    0.221969
2    0.750000
3    0.221969
4    0.309091
Name: product_category, dtype: float64

## product_subcategory

### 5fold te encoding

In [334]:
print(df.product_subcategory.unique())
len(df.product_subcategory.unique())

[nan 'Others' 'Digital Signage or Commercial TVs' 'TR3DJ Series'
 'Transparent OLED Signage' 'Interactive Digital Board'
 'Curvable OLED Signage' 'Transparent LED Film' 'High Brightness Series'
 '55" 700 nits FHD 0.44mm Even Bezel Video Wall'
 '55" 500 nits FHD 0.44mm Even Bezel Video Wall']


11

In [335]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Create a new column for the target-encoded 'product_category'
df['product_subcategory_te'] = np.nan

# Apply K-fold target encoding
for train_idx, val_idx in kf.split(df):
    # Split the data
    df_train, df_val = df.iloc[train_idx], df.iloc[val_idx]
    # Compute the target mean for each category in 'product_category' on the training data
    means = df_train.groupby('product_subcategory')['is_converted'].mean()
    # Map the computed means to the validation set
    df.loc[val_idx, 'product_subcategory_te'] = df_val['product_subcategory'].map(means)

# Fill missing values for any 'product_category' not seen in the training folds
global_mean = df['is_converted'].mean()
df['product_subcategory_te'].fillna(global_mean, inplace=True)

In [336]:
df['product_subcategory'] = df['product_subcategory_te']
df.drop('product_subcategory_te', axis=1, inplace=True)
print(len(df.columns))
df.columns

31


Index(['id', 'bant_submit', 'customer_country', 'business_unit',
       'com_reg_ver_win_rate', 'customer_idx', 'customer_type',
       'historical_existing_cnt', 'id_strategic_ver', 'it_strategic_ver',
       'idit_strategic_ver', 'customer_job', 'lead_desc_length',
       'inquiry_type', 'product_category', 'product_subcategory',
       'product_modelname', 'customer_country.1', 'customer_position',
       'response_corporate', 'expected_timeline', 'ver_cus', 'ver_pro',
       'ver_win_rate_x', 'ver_win_ratio_per_bu', 'business_area',
       'business_subarea', 'lead_owner', 'is_converted',
       'enterprise_Enterprise', 'enterprise_SMB'],
      dtype='object')

## product_modelname

### 5fold te encoding

In [337]:
print(df.product_modelname.unique())
len(df.product_modelname.unique())

[nan 'Others' '55VH7J-H' '65TR3DJ' '55EW5TK-A' '55EF5F-L' 'LAT140'
 'LWBC039' '55VSH7J' '75TR3DJ' '55VSM5J']


11

In [338]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Create a new column for the target-encoded 'product_category'
df['product_modelname_te'] = np.nan

# Apply K-fold target encoding
for train_idx, val_idx in kf.split(df):
    # Split the data
    df_train, df_val = df.iloc[train_idx], df.iloc[val_idx]
    # Compute the target mean for each category in 'product_category' on the training data
    means = df_train.groupby('product_modelname')['is_converted'].mean()
    # Map the computed means to the validation set
    df.loc[val_idx, 'product_modelname_te'] = df_val['product_modelname'].map(means)

# Fill missing values for any 'product_category' not seen in the training folds
global_mean = df['is_converted'].mean()
df['product_modelname_te'].fillna(global_mean, inplace=True)

In [339]:
df['product_modelname'] = df['product_modelname_te']
df.drop('product_modelname_te', axis=1, inplace=True)
print(len(df.columns))
df.columns

31


Index(['id', 'bant_submit', 'customer_country', 'business_unit',
       'com_reg_ver_win_rate', 'customer_idx', 'customer_type',
       'historical_existing_cnt', 'id_strategic_ver', 'it_strategic_ver',
       'idit_strategic_ver', 'customer_job', 'lead_desc_length',
       'inquiry_type', 'product_category', 'product_subcategory',
       'product_modelname', 'customer_country.1', 'customer_position',
       'response_corporate', 'expected_timeline', 'ver_cus', 'ver_pro',
       'ver_win_rate_x', 'ver_win_ratio_per_bu', 'business_area',
       'business_subarea', 'lead_owner', 'is_converted',
       'enterprise_Enterprise', 'enterprise_SMB'],
      dtype='object')

## customer_position

### 5fold te

In [340]:
print(df.customer_position.unique())
len(df.customer_position.unique())

['none' 'manager' 'others' 'associate/analyst' 'director' 'entry level'
 'ceo/founder' 'partner' 'c-level executive' 'trainee' 'vice president'
 'intern' 'educator' 'other' 'customer'
 'academic coordinator/ post graduate teacher (accountancy, business studies)/ tgt (ict)'
 'assistant professor' 'math and physics teacher' 'pgt chemistry'
 'director cum faculty at gaining apex coaching centre' 'professor'
 'co-founder' 'asst prof.']


23

In [341]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

df['customer_position_te'] = np.nan

for train_idx, val_idx in kf.split(df):
    X_train, X_val = df.iloc[train_idx], df.iloc[val_idx]
    means = X_train.groupby('customer_position')['is_converted'].mean()
    df.loc[val_idx, 'customer_position_te'] = X_val['customer_position'].map(means)

global_mean = df['is_converted'].mean()
df['customer_position_te'].fillna(global_mean, inplace=True)

In [342]:
df['customer_position'] = df['customer_position_te']
df.drop('customer_position_te', axis=1, inplace=True)
print(len(df.columns))
df.columns

31


Index(['id', 'bant_submit', 'customer_country', 'business_unit',
       'com_reg_ver_win_rate', 'customer_idx', 'customer_type',
       'historical_existing_cnt', 'id_strategic_ver', 'it_strategic_ver',
       'idit_strategic_ver', 'customer_job', 'lead_desc_length',
       'inquiry_type', 'product_category', 'product_subcategory',
       'product_modelname', 'customer_country.1', 'customer_position',
       'response_corporate', 'expected_timeline', 'ver_cus', 'ver_pro',
       'ver_win_rate_x', 'ver_win_ratio_per_bu', 'business_area',
       'business_subarea', 'lead_owner', 'is_converted',
       'enterprise_Enterprise', 'enterprise_SMB'],
      dtype='object')

## response_corporate

## 10fold te

In [343]:
print(df.response_corporate.unique())
len(df.response_corporate.unique())

['LGESP' 'LGEUS' 'LGEGF' 'LGEDG' 'LGEEG' 'LGEFS' 'LGEIN' 'LGEIL' 'LGEIS'
 'LGEAR' 'LGETK' 'LGESJ' 'LGEPH' 'LGEPR' 'LGEVH' 'LGEAP' 'LGECL' 'LGECB'
 'LGEHK' 'LGEMS' 'LGETH' 'LGEMK' 'LGEBN' 'LGEPS' 'LGEUK' 'LGESA' 'LGESL'
 'LGEYK' 'LGEAG' 'LGEEF' 'LGECH' 'LGECI' 'LGEAF' 'LGEPL' 'LGEES' 'LGEHS'
 'LGEAS' 'LGELF' 'LGERO' 'LGEPT' 'LGEML']


41

In [344]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Create a new column for the target-encoded 'response_corporate'
df['response_corporate_te'] = np.nan

for train_idx, val_idx in kf.split(df):
    # Split the data into training and validation sets
    X_train, X_val = df.iloc[train_idx], df.iloc[val_idx]
    
    # Compute the target mean for each category in 'response_corporate' on the training data
    means = X_train.groupby('response_corporate')['is_converted'].mean()
    
    # Map the computed means to the validation set
    df.loc[val_idx, 'response_corporate_te'] = X_val['response_corporate'].map(means)

# Fill missing values for any 'response_corporate' not seen in the training folds with the global mean
global_mean = df['is_converted'].mean()
df['response_corporate_te'].fillna(global_mean, inplace=True)

# Verify the changes
print(df[['response_corporate', 'response_corporate_te']].head())

  response_corporate  response_corporate_te
0              LGESP               0.251225
1              LGEUS               0.388191
2              LGEGF               0.139344
3              LGEUS               0.388265
4              LGESP               0.257282


In [345]:
df['response_corporate'] = df['response_corporate_te']
df.drop('response_corporate_te', axis=1, inplace=True)
print(len(df.columns))
df.columns

31


Index(['id', 'bant_submit', 'customer_country', 'business_unit',
       'com_reg_ver_win_rate', 'customer_idx', 'customer_type',
       'historical_existing_cnt', 'id_strategic_ver', 'it_strategic_ver',
       'idit_strategic_ver', 'customer_job', 'lead_desc_length',
       'inquiry_type', 'product_category', 'product_subcategory',
       'product_modelname', 'customer_country.1', 'customer_position',
       'response_corporate', 'expected_timeline', 'ver_cus', 'ver_pro',
       'ver_win_rate_x', 'ver_win_ratio_per_bu', 'business_area',
       'business_subarea', 'lead_owner', 'is_converted',
       'enterprise_Enterprise', 'enterprise_SMB'],
      dtype='object')

## expected_timeline

## 5fold te

In [346]:
print(df.expected_timeline.unique())
len(df.expected_timeline.unique())

['etc' 'less than 3 months' 'more than a year' '6 months ~ 9 months'
 '3 months ~ 6 months' '9 months ~ 1 year']


6

In [347]:
# Assuming df is your DataFrame and 'is_converted' is the target variable
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Create a new column for the target-encoded 'expected_timeline'
df['expected_timeline_te'] = np.nan

for train_idx, val_idx in kf.split(df):
    # Split the data
    X_train, X_val = df.iloc[train_idx], df.iloc[val_idx]
    
    # Compute the target mean for 'expected_timeline' on the training data
    means = X_train.groupby('expected_timeline')['is_converted'].mean()
    
    # Map the computed means to the validation set
    df.loc[val_idx, 'expected_timeline_te'] = X_val['expected_timeline'].map(means)

# Fill missing values for any 'expected_timeline' not seen in the training folds with the global mean
global_mean = df['is_converted'].mean()
df['expected_timeline_te'].fillna(global_mean, inplace=True)

# Verify the changes
print(df[['expected_timeline', 'expected_timeline_te']].head())

    expected_timeline  expected_timeline_te
0                 etc              0.221014
1                 etc              0.217617
2  less than 3 months              0.228758
3    more than a year              0.210280
4  less than 3 months              0.228096


In [348]:
df['expected_timeline'] = df['expected_timeline_te']
df.drop('expected_timeline_te', axis=1, inplace=True)
print(len(df.columns))
df.columns

31


Index(['id', 'bant_submit', 'customer_country', 'business_unit',
       'com_reg_ver_win_rate', 'customer_idx', 'customer_type',
       'historical_existing_cnt', 'id_strategic_ver', 'it_strategic_ver',
       'idit_strategic_ver', 'customer_job', 'lead_desc_length',
       'inquiry_type', 'product_category', 'product_subcategory',
       'product_modelname', 'customer_country.1', 'customer_position',
       'response_corporate', 'expected_timeline', 'ver_cus', 'ver_pro',
       'ver_win_rate_x', 'ver_win_ratio_per_bu', 'business_area',
       'business_subarea', 'lead_owner', 'is_converted',
       'enterprise_Enterprise', 'enterprise_SMB'],
      dtype='object')

## business_area

### 5fold te

In [349]:
print(df.business_area.unique())
len(df.business_area.unique())

['retail' 'transportation' 'hospital & health care' 'corporate / office'
 nan 'residential (home)' 'education' 'factory' 'special purpose'
 'hotel & accommodation' 'public facility' 'government department'
 'power plant / renewable energy']


13

In [350]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Create a new column for the target-encoded 'business_area'
df['business_area_te'] = np.nan

for train_idx, val_idx in kf.split(df):
    # Split the data into training and validation sets
    X_train, X_val = df.iloc[train_idx], df.iloc[val_idx]
    
    # Compute the target mean for each category in 'business_area' on the training data
    means = X_train.groupby('business_area')['is_converted'].mean()
    
    # Map the computed means to the validation set
    df.loc[val_idx, 'business_area_te'] = X_val['business_area'].map(means)

# Fill missing values for any 'business_area' not seen in the training folds with the global mean
global_mean = df['is_converted'].mean()
df['business_area_te'].fillna(global_mean, inplace=True)

# Verify the changes
print(df[['business_area', 'business_area_te']].head())

            business_area  business_area_te
0                  retail          0.248996
1          transportation          0.532544
2  hospital & health care          0.339623
3                  retail          0.261224
4      corporate / office          0.331897


In [351]:
df['business_area'] = df['business_area_te']
df.drop('business_area_te', axis=1, inplace=True)
print(len(df.columns))
df.columns

31


Index(['id', 'bant_submit', 'customer_country', 'business_unit',
       'com_reg_ver_win_rate', 'customer_idx', 'customer_type',
       'historical_existing_cnt', 'id_strategic_ver', 'it_strategic_ver',
       'idit_strategic_ver', 'customer_job', 'lead_desc_length',
       'inquiry_type', 'product_category', 'product_subcategory',
       'product_modelname', 'customer_country.1', 'customer_position',
       'response_corporate', 'expected_timeline', 'ver_cus', 'ver_pro',
       'ver_win_rate_x', 'ver_win_ratio_per_bu', 'business_area',
       'business_subarea', 'lead_owner', 'is_converted',
       'enterprise_Enterprise', 'enterprise_SMB'],
      dtype='object')

## business_subarea

In [352]:
print(df.business_subarea.unique())
len(df.business_subarea.unique())

['Electronics & Telco' 'Others' 'General Hospital' nan 'Engineering'
 'Banking' 'Hospital' 'K12 Kindergarten & Schools' 'Construction'
 'Advertising' 'Higher Education (College & University)'
 'Manufacturing Factory / Plant' 'Other Stores' 'Fashion'
 'Telecommunication' 'Mixed-use (Multi Complex)' 'Institute & Academy'
 'Fitness' 'IT/Software' 'Restaurant' 'Cosmetics' 'Healthcare' 'Hotel'
 'Military' 'Clinic' 'Manufacturing' 'Apartment' 'Townhouse' 'Resort'
 'Finance' 'Religious Facility' 'Broadcasting & Media' 'Holdings'
 'Villa / Single-Family Home' 'Power Plant' 'Developer/Property'
 'Renewable Energy' 'Entertainment' 'Sports Entertainment'
 'Telecom Base Station / Data, Call' 'Hyper Market & Grocery'
 'Bus Terminal' 'Luxury(Watch/Jewelry Shop)' 'Logistics' 'Gas Station'
 'Energy' 'General Government Office' 'Outdoor Advertisement' 'Investment'
 'Shopping Mall' 'Energy Storage & Saving' 'F&B(Food and Beverage)'
 'Exhibition / Convention Center' 'Consulting' 'Insurance'
 'QSR(Quick S

81

In [353]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Create a new column for the target-encoded 'business_area'
df['business_subarea_te'] = np.nan

for train_idx, val_idx in kf.split(df):
    # Split the data into training and validation sets
    X_train, X_val = df.iloc[train_idx], df.iloc[val_idx]
    
    # Compute the target mean for each category in 'business_area' on the training data
    means = X_train.groupby('business_subarea')['is_converted'].mean()
    
    # Map the computed means to the validation set
    df.loc[val_idx, 'business_subarea_te'] = X_val['business_subarea'].map(means)

# Fill missing values for any 'business_area' not seen in the training folds with the global mean
global_mean = df['is_converted'].mean()
df['business_subarea_te'].fillna(global_mean, inplace=True)

# Verify the changes
print(df[['business_subarea', 'business_subarea_te']].head())

      business_subarea  business_subarea_te
0  Electronics & Telco             0.192308
1               Others             0.441065
2     General Hospital             0.200000
3                  NaN             0.221969
4          Engineering             0.275862


In [354]:
df['business_subarea'] = df['business_subarea_te']
df.drop('business_subarea_te', axis=1, inplace=True)
print(len(df.columns))
df.columns

31


Index(['id', 'bant_submit', 'customer_country', 'business_unit',
       'com_reg_ver_win_rate', 'customer_idx', 'customer_type',
       'historical_existing_cnt', 'id_strategic_ver', 'it_strategic_ver',
       'idit_strategic_ver', 'customer_job', 'lead_desc_length',
       'inquiry_type', 'product_category', 'product_subcategory',
       'product_modelname', 'customer_country.1', 'customer_position',
       'response_corporate', 'expected_timeline', 'ver_cus', 'ver_pro',
       'ver_win_rate_x', 'ver_win_ratio_per_bu', 'business_area',
       'business_subarea', 'lead_owner', 'is_converted',
       'enterprise_Enterprise', 'enterprise_SMB'],
      dtype='object')

## checking

In [ ]:
# replace the original column with the target encoded column, but the order matters

df = df_encoded
print(len(df.columns))
df.columns

In [355]:
# Select columns that are not of numeric types (int or float)
non_numeric_columns = df.select_dtypes(exclude=['int64', 'int32', 'float64']).columns

# Check if there are any non-numeric columns
if len(non_numeric_columns) > 0:
    print(f"Non-numeric columns found: {list(non_numeric_columns)}")
else:
    print("All columns are of int or float type.")

Non-numeric columns found: ['enterprise_Enterprise', 'enterprise_SMB']


In [356]:
df.enterprise_Enterprise.head()

#convert df.enterprise_Enterprise to int
df['enterprise_Enterprise'] = df['enterprise_Enterprise'].astype('int64')
df['enterprise_SMB'] = df['enterprise_SMB'].astype('int64')

df['enterprise_Enterprise'].head()
df['enterprise_SMB'].head()

0    0
1    1
2    1
3    0
4    0
Name: enterprise_SMB, dtype: int64

In [357]:
non_numeric_columns = df.select_dtypes(exclude=['int64', 'int32', 'float64']).columns

# Check if there are any non-numeric columns
if len(non_numeric_columns) > 0:
    print(f"Non-numeric columns found: {list(non_numeric_columns)}")
else:
    print("All columns are of int or float type.")

All columns are of int or float type.


In [358]:
df.to_csv('encoded_submission_raw.csv', index=False)